In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('../data/linux_gaming_SteamDeck_merged_title_data.csv')

In [3]:
df.head()

,title_id,title,subreddit
0,12xr0d6,Valve moves ELDEN RING back to Proton 7 as it ...,linux_gaming
1,12xbjm3,Improved AMD Color Management Being Worked On ...,linux_gaming
2,12xtiug,My first solo made game Lichenvale is out on S...,linux_gaming
3,12xrnss,Fedora 39 Looks To Boost vm.max_map_count To H...,linux_gaming
4,12wu74d,Roblox intentionally blocking Linux with Wine ...,linux_gaming


In [4]:
df.shape

(2864, 3)